In [0]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
import torch.nn.functional as F
import numpy as np
import torch.optim as optim

In [3]:
!unzip preprocessed_data.zip

Archive:  preprocessed_data.zip
   creating: preprocessed_data/
  inflating: preprocessed_data/train_questions.npy  
  inflating: preprocessed_data/val_questions.npy  
  inflating: preprocessed_data/train_image_features.npy  
  inflating: preprocessed_data/answer_tokens_top1k.json  
  inflating: preprocessed_data/val_dataset_1k.json  
  inflating: preprocessed_data/val_dataset_1k_tokenised.json  
  inflating: preprocessed_data/combined_val_input.npy  
  inflating: preprocessed_data/val_image_features.npy  
  inflating: preprocessed_data/train_dataset_5k.json  
  inflating: preprocessed_data/train_dataset_5k_tokenised.json  
   creating: preprocessed_data/.ipynb_checkpoints/
  inflating: preprocessed_data/.ipynb_checkpoints/Simple_CNN+LSTM_Model-checkpoint.ipynb  
  inflating: preprocessed_data/val_answers.npy  
  inflating: preprocessed_data/train_answers.npy  
   creating: preprocessed_data/6B.50.dat/
  inflating: preprocessed_data/6B.50.dat/__attrs__  
   creating: preprocessed_data/

In [4]:
!wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

--2020-04-02 15:06:08--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2020-04-02 15:06:08--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2020-04-02 15:06:08--  http://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [applic

In [5]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
  inflating: glove.6B.50d.txt        


In [6]:
!pip install bcolz

     |████████████████████████████████| 1.5MB 4.7MB/s 
  Created wheel for bcolz: filename=bcolz-1.2.1-cp36-cp36m-linux_x86_64.whl size=2666373 sha256=5f924732a4f7ed7a0a383235c4ce8684d97d4db550c6bdb560a4a4a0d690e0a2
  Stored in directory: /root/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
Successfully built bcolz


In [3]:
image_features_train = np.load("preprocessed_data/train_image_features.npy")
questions_train = np.load("preprocessed_data/train_questions.npy")
answers_train = np.load("preprocessed_data/train_answers.npy")
print(image_features_train.shape)
print(questions_train.shape)
print(answers_train.shape)

image_features_val = np.load("preprocessed_data/val_image_features.npy")
questions_val = np.load("preprocessed_data/val_questions.npy")
answers_val = np.load("preprocessed_data/val_answers.npy")
print(image_features_val.shape)
print(questions_val.shape)
print(answers_val.shape)

(4269, 512)
(4269, 25)
(4269, 1)
(837, 512)
(837, 25)
(837, 1)


In [3]:
import bcolz
import pickle

words = []
idx = 0
word2idx = {}
glove_path = ""
vectors = bcolz.carray(np.zeros(1), rootdir=f'6B.50.dat', mode='w')

with open(f'glove.6B.300d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)

print(vectors.shape)

(120000301,)


In [0]:
vectors = bcolz.carray(vectors[1:].reshape((400001, 300)), rootdir=f'6B.300.dat', mode='w')
vectors.flush()
pickle.dump(words, open(f'6B.300_words.pkl', 'wb'))
pickle.dump(word2idx, open(f'6B.300_idx.pkl', 'wb'))

In [0]:
import bcolz
import pickle 

vectors = bcolz.open(f'6B.300.dat')[:]
words = pickle.load(open(f'6B.300_words.pkl', 'rb'))
word2idx = pickle.load(open(f'6B.300_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

In [7]:
matrix_len = 400001
weights_matrix = np.zeros((matrix_len, 300))
words_found = 0

for i, word in enumerate(glove.keys()):
    try: 
        weights_matrix[i] = glove[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))

weights_matrix=torch.from_numpy(weights_matrix)
print(weights_matrix.size())

torch.Size([400001, 300])


In [0]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

In [0]:
class AttentionNetwork(nn.Module):

    def __init__(self, encoder_dim, decoder_dim, attention_dim):
    
        super(AttentionNetwork, self).__init__()
        self.encoder_att = nn.Linear(encoder_dim, attention_dim)  
        self.decoder_att = nn.Linear(encoder_dim, attention_dim) 
        self.full_att = nn.Linear(attention_dim, encoder_dim)  
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)  

    def forward(self, encoder_out, decoder_hidden):
        att1 = self.encoder_att(encoder_out)  
        att2 = self.decoder_att(decoder_hidden)  
        att = self.full_att(self.relu(att1 + att2))  
        alpha = self.softmax(att)  
        attention_weighted_encoding = (encoder_out * alpha) 
        return attention_weighted_encoding, alpha


In [0]:
class DecodeAttentionMaps(nn.Module):

  def __init__(self, attention_dim, embed_dim, encoder_dim, max_length, vocab_size, dropout):
    super(DecodeAttentionMaps, self).__init__()
    self.attention_dim=attention_dim
    self.embed_dim=embed_dim
    self.encoder_dim=encoder_dim
    self.max_length=max_length
    self.vocab_size=vocab_size
    self.dropout=nn.Dropout(dropout)

    self.embedding = nn.Embedding(vocab_size, embed_dim)
    self.tanh=nn.Tanh()
    self.question_lstm = nn.LSTM(embed_dim, 1024, num_layers=1, batch_first=True)

    self.attention=AttentionNetwork(encoder_dim, max_length, attention_dim)
    #self.image_linear=nn.Linear(encoder_dim, embed_dim)
    #self.embedding, num_embeddings, embedding_dim=create_emb_layer(weights_matrix, True)
    self.relu=nn.ReLU()
    self.linear1=nn.Linear(1536, 1024)
    self.linear2=nn.Linear(1024, encoder_dim)
    self.fc=nn.Linear(encoder_dim, 1000)
    self.softmax=nn.Softmax(dim=1)
    self.sigmoid=nn.Sigmoid()
    self.gru = nn.GRU(embed_dim, encoder_dim, num_layers=1, batch_first=True)
  
  def forward(self, images, questions):
    question_embeddings = self.dropout(self.tanh(self.embedding(questions.type(torch.long))))  # (batch_size, question_length, embed_dim)
    output, (hidden, cell) = self.question_lstm(question_embeddings)
    hidden = hidden.squeeze(0)
    concatenated_features = self.dropout(torch.cat((images, hidden), dim=1))
    features=self.dropout(self.relu(self.linear1(concatenated_features)))
    attention_weighted_encoding = self.softmax(self.linear2(features))
    #attention_weighted_images=(alpha*images)
    x=F.relu(self.fc(attention_weighted_encoding))
    return self.softmax(x)
    '''
    out, hidden = self.gru(question_embeddings)
    hidden=self.sigmoid(self.linear1(hidden))
    images=images
    i_q_embed=hidden.squeeze(0)*images
    #y = torch.sqrt(F.relu(i_q_embed)) - torch.sqrt(F.relu(-i_q_embed))
    #y=self.dropout(y)
    alpha=self.softmax(i_q_embed)
    #y=y/torch.norm(y,p=2,dim=2).unsqueeze(2)
    #y=torch.exp(self.linear2(F.relu(self.linear1(y))))

    #alpha= y/torch.sum(y,axis=2).unsqueeze(2)
    attention_weighted_images=(alpha*images).squeeze(0)
    x=F.relu(self.fc(attention_weighted_images))
    return self.softmax(x)
    #hidden=hidden.squeeze(0)
    #attention_weighted_encoding, alpha = self.attention(images.type(torch.float), hidden.type(torch.float))
    #x = F.relu(self.fc(attention_weighted_encoding))
    #return self.softmax(x)
    
    '''
    

In [0]:
images=torch.tensor(image_features_train, dtype=torch.float)
questions=torch.tensor(questions_train)
max_length=questions.size(-1)
encoder_dim=images.size(-1)
vocab_size=len(word2idx)
attention_dim=512
embed_dim=300

In [0]:
model=DecodeAttentionMaps(attention_dim, embed_dim, encoder_dim, max_length, vocab_size, 0.5)

In [0]:
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [0]:
def get_accuracy(predictions, labels):
  predictions = torch.max(predictions, axis=1)[1]
  # print(predictions)
  # print(labels)
  # print(predictions.size())
  # print(labels.size())
  ab = torch.abs(predictions-labels)
  # print(ab.size())
  ab = ab.detach().numpy()
  mn = np.minimum(ab, 1)
  eq = 1-mn
  # eq = 1 - torch.min(torch.abs(predictions-labels), 1)
  correct = np.sum(eq)
  # print(correct)
  total = eq.shape[0]
  return correct, total

In [15]:
from torch.utils import data
tensor_x = torch.tensor(image_features_train, dtype=torch.float)
tensor_y = torch.tensor(questions_train)
tensor_z = torch.Tensor(answers_train).flatten()
tensor_z = tensor_z.type(torch.long)
print(tensor_x.size())
print(tensor_y.size())
print(tensor_z.size())
print(tensor_x.dtype)
print(tensor_y.dtype)
print(tensor_z.dtype)
trainset = data.TensorDataset(tensor_x,tensor_y,tensor_z)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1000, shuffle=True)

torch.Size([4269, 512])
torch.Size([4269, 25])
torch.Size([4269])
torch.float32
torch.float64
torch.int64


In [16]:
from torch.utils import data
tensor_x = torch.tensor(image_features_val,dtype=torch.float)
tensor_y = torch.tensor(questions_val)
tensor_z = torch.Tensor(answers_val).flatten()
tensor_z = tensor_z.type(torch.long)
print(tensor_x.size())
print(tensor_y.size())
print(tensor_z.size())
print(tensor_x.dtype)
print(tensor_y.dtype)
print(tensor_z.dtype)
valset = data.TensorDataset(tensor_x,tensor_y,tensor_z)
valloader = torch.utils.data.DataLoader(valset, batch_size=1000, shuffle=True)

torch.Size([837, 512])
torch.Size([837, 25])
torch.Size([837])
torch.float32
torch.float64
torch.int64


In [0]:
import matplotlib.pyplot as plt 
import torch.nn.functional as F
train_loss=[]
val_loss=[]

for epoch in range(60):  # loop over the dataset multiple times

    running_loss = 0.0
    correct = 0
    total = 0
    total_loss = 0.0
    model.train()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, questions, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs, questions)
        batch_correct, batch_total = get_accuracy(outputs, labels)
        correct += batch_correct
        total += batch_total
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        # if i % 1 == 0:    # print every 2000 mini-batches
        #     print('[%d, %5d] loss: %.3f' %
        #           (epoch + 1, i + 1, running_loss / 2000))
        #     running_loss = 0.0
        total_loss += running_loss
        running_loss = 0.0

    val_correct = 0
    val_total = 0
    model.eval()
    with torch.no_grad():
      for i, data in enumerate(valloader, 0):
          # get the inputs; data is a list of [inputs, labels]
          inputs, questions, labels = data
          # forward + backward + optimize
          outputs = model(inputs, questions)
          batch_correct, batch_total = get_accuracy(outputs, labels)
          val_correct += batch_correct
          val_total += batch_total
          
    print("Epoch: ",epoch," Loss: ",total_loss," Train-Accuracy: ", correct/total," Val-Accuracy: ",val_correct/val_total)
  

print('Finished Training')

Epoch:  0  Loss:  34.538620948791504  Train-Accuracy:  0.02506441789646287  Val-Accuracy:  0.03225806451612903
Epoch:  1  Loss:  34.538557052612305  Train-Accuracy:  0.030217849613492623  Val-Accuracy:  0.03225806451612903
Epoch:  2  Loss:  34.538429260253906  Train-Accuracy:  0.030217849613492623  Val-Accuracy:  0.03225806451612903
Epoch:  3  Loss:  34.5381383895874  Train-Accuracy:  0.030217849613492623  Val-Accuracy:  0.03225806451612903
Epoch:  4  Loss:  34.537734031677246  Train-Accuracy:  0.030217849613492623  Val-Accuracy:  0.03225806451612903


In [0]:
for param_group in optimizer.param_groups:
        param_group['lr'] = 0.0001

In [0]:
for epoch in range(17):  # loop over the dataset multiple times

    running_loss = 0.0
    correct = 0
    total = 0
    total_loss = 0.0
    model.train()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, questions, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs, questions)
        batch_correct, batch_total = get_accuracy(outputs, labels)
        correct += batch_correct
        total += batch_total
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        # if i % 1 == 0:    # print every 2000 mini-batches
        #     print('[%d, %5d] loss: %.3f' %
        #           (epoch + 1, i + 1, running_loss / 2000))
        #     running_loss = 0.0
        total_loss += running_loss
        running_loss = 0.0

    val_correct = 0
    val_total = 0
    model.eval()
    with torch.no_grad():
      for i, data in enumerate(valloader, 0):
          # get the inputs; data is a list of [inputs, labels]
          inputs, questions, labels = data
          # forward + backward + optimize
          outputs = model(inputs, questions)
          batch_correct, batch_total = get_accuracy(outputs, labels)
          val_correct += batch_correct
          val_total += batch_total
          
    print("Epoch: ",epoch," Loss: ",total_loss," Train-Accuracy: ", correct/total," Val-Accuracy: ",val_correct/val_total)
  

print('Finished Training')

Epoch:  0  Loss:  34.53840446472168  Train-Accuracy:  0.21878660107753573  Val-Accuracy:  0.1911589008363202
Epoch:  1  Loss:  34.53840446472168  Train-Accuracy:  0.22019208245490748  Val-Accuracy:  0.1911589008363202
Epoch:  2  Loss:  34.538405895233154  Train-Accuracy:  0.22019208245490748  Val-Accuracy:  0.1911589008363202
Epoch:  3  Loss:  34.538392066955566  Train-Accuracy:  0.22089482314359335  Val-Accuracy:  0.1911589008363202
Epoch:  4  Loss:  34.53838777542114  Train-Accuracy:  0.22112907003982196  Val-Accuracy:  0.19474313022700118
Epoch:  5  Loss:  34.538395404815674  Train-Accuracy:  0.2213633169360506  Val-Accuracy:  0.19474313022700118
Epoch:  6  Loss:  34.538389682769775  Train-Accuracy:  0.2213633169360506  Val-Accuracy:  0.1959378733572282
Epoch:  7  Loss:  34.53839445114136  Train-Accuracy:  0.22183181072850786  Val-Accuracy:  0.1959378733572282
Epoch:  8  Loss:  34.53838300704956  Train-Accuracy:  0.22159756383227922  Val-Accuracy:  0.1959378733572282
Epoch:  9  Loss

In [0]:
for epoch in range(34):  # loop over the dataset multiple times

    running_loss = 0.0
    correct = 0
    total = 0
    total_loss = 0.0
    model.train()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, questions, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs, questions)
        batch_correct, batch_total = get_accuracy(outputs, labels)
        correct += batch_correct
        total += batch_total
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        # if i % 1 == 0:    # print every 2000 mini-batches
        #     print('[%d, %5d] loss: %.3f' %
        #           (epoch + 1, i + 1, running_loss / 2000))
        #     running_loss = 0.0
        total_loss += running_loss
        running_loss = 0.0

    val_correct = 0
    val_total = 0
    model.eval()
    with torch.no_grad():
      for i, data in enumerate(valloader, 0):
          # get the inputs; data is a list of [inputs, labels]
          inputs, questions, labels = data
          # forward + backward + optimize
          outputs = model(inputs, questions)
          batch_correct, batch_total = get_accuracy(outputs, labels)
          val_correct += batch_correct
          val_total += batch_total
          
    print("Epoch: ",epoch," Loss: ",total_loss," Train-Accuracy: ", correct/total," Val-Accuracy: ",val_correct/val_total)
  

print('Finished Training')

Epoch:  0  Loss:  34.538328647613525  Train-Accuracy:  0.2223003045209651  Val-Accuracy:  0.1971326164874552
Epoch:  1  Loss:  34.53831624984741  Train-Accuracy:  0.2223003045209651  Val-Accuracy:  0.1971326164874552
Epoch:  2  Loss:  34.53829526901245  Train-Accuracy:  0.2223003045209651  Val-Accuracy:  0.1971326164874552
Epoch:  3  Loss:  34.53827142715454  Train-Accuracy:  0.2223003045209651  Val-Accuracy:  0.1971326164874552
Epoch:  4  Loss:  34.53825855255127  Train-Accuracy:  0.2223003045209651  Val-Accuracy:  0.1971326164874552
Epoch:  5  Loss:  34.53824853897095  Train-Accuracy:  0.2223003045209651  Val-Accuracy:  0.1971326164874552
Epoch:  6  Loss:  34.53822469711304  Train-Accuracy:  0.2223003045209651  Val-Accuracy:  0.1971326164874552
Epoch:  7  Loss:  34.538185119628906  Train-Accuracy:  0.2223003045209651  Val-Accuracy:  0.1971326164874552
Epoch:  8  Loss:  34.53818464279175  Train-Accuracy:  0.22206605762473647  Val-Accuracy:  0.1971326164874552
Epoch:  9  Loss:  34.5381

KeyboardInterrupt: ignored